In [1]:
from bs4 import BeautifulSoup
import urllib3
import pandas as pd
import numpy as np

In [2]:
def get_stores(url):
    http = urllib3.PoolManager()
    r = http.request('GET', url)
    soup = BeautifulSoup(r.data, 'html.parser')
    stores_dict = {}
    stores_array = []
    for item_index, item in enumerate(soup.find_all('ul', class_='stores__list')):
        for li in item.find_all('li'):
            for div in li.find_all('div'):
                for p_index, p in enumerate(div.find_all('p')):
                    stores_dict[div.get('class')[1].split("--is-")[1]] = p.text
                for a in div.find_all('a'):
                    stores_dict['lat'] = a.get('href').split('/')[-1].split(',')[0]
                    stores_dict['lon'] = a.get('href').split('/')[-1].split(',')[1]
            stores_array.append(pd.DataFrame.from_dict(stores_dict, orient='index').T)
    stores_df = pd.concat(stores_array, ignore_index=True)
    return stores_df

In [3]:
dataframes = []
for i in range(1,6):
    dataframes.append((get_stores(f'https://salcobrand.cl/content/servicios/mapa?region_id=15&state_id=375&page={i}')))

In [4]:
result = pd.concat(dataframes, ignore_index=True)
result.drop(columns=['availability-open', 'waittime','availability-closed','parking-closed'], inplace=True)
result.replace(np.nan, 'No disponible', inplace=True)
result

,address,county,region,parking-open,schedule,lat,lon
0,Avda. Apoquindo N° 5855 - 5879,Las condes,METROPOLITANA DE SANTIAGO,Disponible,Domingo: 10:00 - 21:00,-33.4088948,-70.56916
1,Avda. Tomas Moro N° 20,Las condes,METROPOLITANA DE SANTIAGO,Disponible,Domingo: 10:00-21:00,-33.4083914,-70.55182359999999
2,Avda. Las Condes N° 12145,Las condes,METROPOLITANA DE SANTIAGO,Disponible,Lunes a Domingo: 00:00-23:59,-33.3739131,-70.5202523
3,Avda. Las Condes N° 8231,Las condes,METROPOLITANA DE SANTIAGO,Disponible,Domingo: 10:00 -19:00,-33.3983044,-70.551147
4,Avda. Cuarto Centenario N° 933,Las condes,METROPOLITANA DE SANTIAGO,Disponible,Domingo: Cerrado,-33.417108,-70.5544287
5,Avda. El Bosque Norte N° 198,Las condes,METROPOLITANA DE SANTIAGO,Disponible,Sábado y Domingo: Cerrado,-33.4149972,-70.6014881
6,Avda. Colón N° 6064,Las condes,METROPOLITANA DE SANTIAGO,Disponible,Sábado y Domingo: 09:00 - 22:00,-33.4231818,-70.5311935
7,Avda. Padre Hurtado N° 40,Las condes,METROPOLITANA DE SANTIAGO,Disponible,Lunes a Domingo: 00:00-23:59,-33.3941372,-70.5458286
8,Avda. Francisco Bilbao N°8750 Local 3048,Las condes,METROPOLITANA DE SANTIAGO,Disponible,Lunes a Domingo: 08:30 - 20:00,-33.4287187,-70.5405628
9,Avda. Américo Vespucio N° 1463,Las condes,METROPOLITANA DE SANTIAGO,Disponible,Domingo: 10:00 - 21:00,-33.4006416,-70.58692289999999


In [5]:
result.to_csv('stores.csv', index=False)

In [26]:
import folium
import folium.plugins
map = folium.Map(location=[-33.406819, -70.547657], zoom_start=13, tiles='CartoDB Positron')
result.apply(lambda x: folium.Marker(location=[x['lat'], x['lon']], popup=x.address).add_to(map), axis=1)
folium.plugins.Fullscreen(position='topright', title='Expandir').add_to(map)
map